In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with open('mixed','r',encoding='utf-8') as file:
    raw0 = ' '.join(file.readlines())
raw0[0:200]

'lkkkk “stag lando\n \n \n Chronicles of Avonlea\n by\n Lucy Maud Montgomery\n Part 1 out of 5\n \n     FullBooks.com homepage\n     Index of Chronicles of Avonlea\n     Next part (2)\n \n \n \n \n \n \n \n \n Typed and '

In [143]:
# [needs improvement]
# remove problem characters
import string
#raw0 = raw0.replace(u'’', u"'").replace('”','"').replace('“','"').replace('—','-').replace("‘","'").replace('Æ','AE')
raw0 = raw0.translate(raw0.maketrans("—’‘Æsæíúáâó‚…ë","-''Asaiuaao,_e" ))
raw0 = raw0.translate(raw0.maketrans('”“','""'))

In [144]:
# remove punctuation and numbers & change letters to lowercase
raw1 = raw0.lower()
raw1 = raw1.translate(raw0.maketrans({'\n':None}))
raw1 = raw1.translate(raw1.maketrans(string.punctuation, ' '*len(string.punctuation)))
raw1 = raw1.translate(raw1.maketrans(string.digits, ' '*len(string.digits)))
raw1 = ''.join(filter(lambda x: x.isspace or x.isalpha, raw1))
raw1[0:2000]

'lkkkk  stag lando   chronicles of avonlea by lucy maud montgomery part   out of        fullbooks com homepage     index of chronicles of avonlea     next part             typed and corrected by kjell nedrelid  last modified              dd mm yyyy  uncorrected version released to internet             this version released             you can freely distribute this text in any form as long as you don t get paid for it  please include this short header       the boys  life of abraham lincoln by helen nicolay part   out of        fullbooks com homepage     index of the boys  life of abraham lincoln     next part             scanned by dianne bean of phoenix  arizona    chronicles of avonlea by lucy maud montgomery part   out of        fullbooks com homepage     index of chronicles of avonlea     previous part        and i ll be as amiable as i know how  but since you have gone to the trouble of getting my supper for me  nancy  you must stay and help me eat it  them strawberries look good

In [145]:
len(raw1)

17068935

In [146]:
# split text into a words list and remove repeating words
raw2 = list(set(raw1.split()))
raw2.sort()
raw2[0:5]

['a', 'aa', 'aaa', 'aah', 'aahed']

In [ ]:
# remove all but the 5-letter words
raw2copy  = raw2.copy()
for word in raw2copy:
    if len(word) != 5:
        raw2.remove(word)
raw2[0:5]

In [ ]:
# a side step to determine letter frequencies only in 5 letter words
five_letter_words_text = ''.join(raw2)
five_letter_words_text[0:100]

In [ ]:
freqs5 = {}
for letter in string.ascii_lowercase:
    freqs5[letter] = float("{:.2f}".format(100*five_letter_words_text.count(letter)/len(five_letter_words_text)))
freqs5

In [ ]:
# create a letter frequency dictionary based on
# http://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html
freqs = {
    'e':12.02,
    't':9.10,
    'a':8.12,
    'o':7.68,
    'i':7.31,
    'n':6.95,
    's':6.28,
    'r':6.02,
    'h':5.92,
    'd':4.32,
    'l':3.98,
    'u':2.88,
    'c':2.71,
    'm':2.61,
    'f':2.30,
    'y':2.11,
    'w':2.09,
    'g':2.03,
    'p':1.82,
    'b':1.49,
    'v':1.11,
    'k':0.69,
    'x':0.17,
    'q':0.11,
    'j':0.10,
    'z':0.07
}

In [ ]:
# compare the 'all words' frequencywith the 'five letter words' frequency
for letter in freqs:
    print(f'{letter} : {round(freqs5[letter]/freqs[letter],3)}')

In [ ]:
# create a 'words' dataframe
words_df = pd.DataFrame(data = {'words':raw2})
words_df.head(2)

In [ ]:
# create a 'frequency score' column
# for each word all letters' frequency values are added to get the word's score

# single & double quotes are to be dealt with
def freq_sum(word):
    score = 0
    for letter in word:
        try:
            score = score + freqs[letter]
        except KeyError:
            pass
    return float("{:.3f}".format(score))

words_df['frequency score'] = words_df['words'].apply(freq_sum)
words_df.head(3)

In [ ]:
# create a 'freq5 score' column but use the freq5 values
def freq5_sum(word):
    score = 0
    for letter in word:
        try:
            score = score + freqs5[letter]
        except KeyError:
            pass
    return float("{:.2f}".format(score))

words_df['freq5 score'] = words_df['words'].apply(freq5_sum)
words_df.head(2)

In [ ]:
# create a 'vowel count' column
# count the number of vowels in the word
vowels = 'aeiou'
def count_vowels(word):
    fraction = 0
    for letter in word:
        if letter in vowels:
            fraction +=1
    return fraction
words_df['vowels count'] = words_df['words'].apply(count_vowels)
words_df.head(2)

In [ ]:
# create a 'repeating letters' column
# return a list of tupples of the form (letter, repetitions)
def repeating_letters(word):
    if len(set(word)) == len(word):
        return 'no repeats'
    repetitions = []
    for letter in word:
        if word.count(letter)>1:
            repetitions.append((letter,word.count(letter)))
    return list(set(repetitions))[0]
            
words_df['repeating letters'] = words_df['words'].apply(repeating_letters)
words_df.head(2)

In [ ]:
# top scoring words
words_df[words_df['repeating letters']=="no repeats"].sort_values('frequency score',ascending = False).head()

In [ ]:
# creating an empty table of all letters and all positions
letters_df = pd.DataFrame(data = np.zeros((5,26), dtype=int), index = [1,2,3,4,5], columns=list(string.ascii_lowercase))

In [ ]:
# count the number of appearances of each letter on each position
for i in range(5):
    for word in words_df.words:
        letters_df.loc[i+1, str(word[i])] += 1

In [ ]:
letters_df

In [ ]:
# normalize for each letter
sums = pd.Series(letters_df.sum(axis=0))
best_pos_for_letter = round(100*letters_df/sums,1)
best_pos_for_letter

In [ ]:
# plot heatmap of the best position for each letter
plt.rcParams["figure.figsize"] = (20,3)

fig, ax = plt.subplots()
im = ax.imshow(best_pos_for_letter, cmap='bone')
ax.set_xticks(np.arange(26), labels=string.ascii_lowercase)
ax.set_yticks(np.arange(5), labels='12345')

# only accent on positions where the appearance is larger than average (100% / 5 = 20%)
for i in range(len('12345')):
    for j in range(len(string.ascii_lowercase)):
        if best_pos_for_letter.iloc[i,j] > 20:
            text = ax.text(j, i, best_pos_for_letter.iloc[i,j], ha="center", va="center", color="r")
        else:
            text = ax.text(j, i, 'x', ha="center", va="center", color="g")

plt.title( "Best position for each letter" )
plt.show()

In [ ]:
# analogical chart for 'best letter for each position'